# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2dfc3a2d00>
├── 'a' --> tensor([[-0.1113, -0.4839,  0.0426],
│                   [ 0.6874,  0.0147,  0.2200]])
└── 'x' --> <FastTreeValue 0x7f2dfc3a2cd0>
    └── 'c' --> tensor([[ 2.0425,  0.5871, -1.6850, -2.0679],
                        [ 0.6888, -0.1030,  0.1280,  1.0964],
                        [ 0.6884, -1.2083,  1.4923, -1.2985]])

In [4]:
t.a

tensor([[-0.1113, -0.4839,  0.0426],
        [ 0.6874,  0.0147,  0.2200]])

In [5]:
%timeit t.a

60.6 ns ± 0.0742 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2dfc3a2d00>
├── 'a' --> tensor([[-1.6021, -0.9565, -0.6676],
│                   [-0.1145,  0.6526,  2.2631]])
└── 'x' --> <FastTreeValue 0x7f2dfc3a2cd0>
    └── 'c' --> tensor([[ 2.0425,  0.5871, -1.6850, -2.0679],
                        [ 0.6888, -0.1030,  0.1280,  1.0964],
                        [ 0.6884, -1.2083,  1.4923, -1.2985]])

In [7]:
%timeit t.a = new_value

58 ns ± 0.0323 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1113, -0.4839,  0.0426],
               [ 0.6874,  0.0147,  0.2200]]),
    x: Batch(
           c: tensor([[ 2.0425,  0.5871, -1.6850, -2.0679],
                      [ 0.6888, -0.1030,  0.1280,  1.0964],
                      [ 0.6884, -1.2083,  1.4923, -1.2985]]),
       ),
)

In [10]:
b.a

tensor([[-0.1113, -0.4839,  0.0426],
        [ 0.6874,  0.0147,  0.2200]])

In [11]:
%timeit b.a

53.6 ns ± 0.054 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.1442, -0.0399,  1.3588],
               [-0.0574, -2.0779, -1.4055]]),
    x: Batch(
           c: tensor([[ 2.0425,  0.5871, -1.6850, -2.0679],
                      [ 0.6888, -0.1030,  0.1280,  1.0964],
                      [ 0.6884, -1.2083,  1.4923, -1.2985]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.186 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

844 ns ± 0.0959 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 40.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 1.61 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 576 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2d4b5ec3a0>
├── 'a' --> tensor([[[-0.1113, -0.4839,  0.0426],
│                    [ 0.6874,  0.0147,  0.2200]],
│           
│                   [[-0.1113, -0.4839,  0.0426],
│                    [ 0.6874,  0.0147,  0.2200]],
│           
│                   [[-0.1113, -0.4839,  0.0426],
│                    [ 0.6874,  0.0147,  0.2200]],
│           
│                   [[-0.1113, -0.4839,  0.0426],
│                    [ 0.6874,  0.0147,  0.2200]],
│           
│                   [[-0.1113, -0.4839,  0.0426],
│                    [ 0.6874,  0.0147,  0.2200]],
│           
│                   [[-0.1113, -0.4839,  0.0426],
│                    [ 0.6874,  0.0147,  0.2200]],
│           
│                   [[-0.1113, -0.4839,  0.0426],
│                    [ 0.6874,  0.0147,  0.2200]],
│           
│                   [[-0.1113, -0.4839,  0.0426],
│                    [ 0.6874,  0.0147,  0.2200]]])
└── 'x' --> <FastTreeValue 0x7f2d4460bd00>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 75.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2d4460bbb0>
├── 'a' --> tensor([[-0.1113, -0.4839,  0.0426],
│                   [ 0.6874,  0.0147,  0.2200],
│                   [-0.1113, -0.4839,  0.0426],
│                   [ 0.6874,  0.0147,  0.2200],
│                   [-0.1113, -0.4839,  0.0426],
│                   [ 0.6874,  0.0147,  0.2200],
│                   [-0.1113, -0.4839,  0.0426],
│                   [ 0.6874,  0.0147,  0.2200],
│                   [-0.1113, -0.4839,  0.0426],
│                   [ 0.6874,  0.0147,  0.2200],
│                   [-0.1113, -0.4839,  0.0426],
│                   [ 0.6874,  0.0147,  0.2200],
│                   [-0.1113, -0.4839,  0.0426],
│                   [ 0.6874,  0.0147,  0.2200],
│                   [-0.1113, -0.4839,  0.0426],
│                   [ 0.6874,  0.0147,  0.2200]])
└── 'x' --> <FastTreeValue 0x7f2d4463bf40>
    └── 'c' --> tensor([[ 2.0425,  0.5871, -1.6850, -2.0679],
                        [ 0.6888, -0.1030,  0.1280,  1.0964],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 117 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

69.7 µs ± 83.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 2.0425,  0.5871, -1.6850, -2.0679],
                       [ 0.6888, -0.1030,  0.1280,  1.0964],
                       [ 0.6884, -1.2083,  1.4923, -1.2985]],
              
                      [[ 2.0425,  0.5871, -1.6850, -2.0679],
                       [ 0.6888, -0.1030,  0.1280,  1.0964],
                       [ 0.6884, -1.2083,  1.4923, -1.2985]],
              
                      [[ 2.0425,  0.5871, -1.6850, -2.0679],
                       [ 0.6888, -0.1030,  0.1280,  1.0964],
                       [ 0.6884, -1.2083,  1.4923, -1.2985]],
              
                      [[ 2.0425,  0.5871, -1.6850, -2.0679],
                       [ 0.6888, -0.1030,  0.1280,  1.0964],
                       [ 0.6884, -1.2083,  1.4923, -1.2985]],
              
                      [[ 2.0425,  0.5871, -1.6850, -2.0679],
                       [ 0.6888, -0.1030,  0.1280,  1.0964],
                       [ 0.6884, -1.2083,  1.4923, -1.2985]],

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 211 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 2.0425,  0.5871, -1.6850, -2.0679],
                      [ 0.6888, -0.1030,  0.1280,  1.0964],
                      [ 0.6884, -1.2083,  1.4923, -1.2985],
                      [ 2.0425,  0.5871, -1.6850, -2.0679],
                      [ 0.6888, -0.1030,  0.1280,  1.0964],
                      [ 0.6884, -1.2083,  1.4923, -1.2985],
                      [ 2.0425,  0.5871, -1.6850, -2.0679],
                      [ 0.6888, -0.1030,  0.1280,  1.0964],
                      [ 0.6884, -1.2083,  1.4923, -1.2985],
                      [ 2.0425,  0.5871, -1.6850, -2.0679],
                      [ 0.6888, -0.1030,  0.1280,  1.0964],
                      [ 0.6884, -1.2083,  1.4923, -1.2985],
                      [ 2.0425,  0.5871, -1.6850, -2.0679],
                      [ 0.6888, -0.1030,  0.1280,  1.0964],
                      [ 0.6884, -1.2083,  1.4923, -1.2985],
                      [ 2.0425,  0.5871, -1.6850, -2.0679],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 179 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 1.43 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
